In [26]:
# 3rd party
import sympy as sp
import numpy as np
from IPython.utils.io import capture_output
from IPython.display import display, Latex
import matplotlib.pyplot as plt

# Our stuff
import mainframe as mf
import nuclei_func as nf
from mainframe import Region

In [27]:
################################################################################
# INPUT DATA
################################################################################
rho_C = 1.7 # We assume is the density of the mixture given the low Uranium content
N_CtoN_U = 10000 # Graphite-to-Uranium density ratio

# Cross sections
molar_mass = nf.mixture([12.01, 235.044], [1 - 1/N_CtoN_U, 1/N_CtoN_U])

macro_abs_core_U = nf.macro(548+105, rho_C, molar_mass)
macro_abs_core_C = nf.macro(0.003, rho_C, molar_mass)
macro_abs_core = nf.mixture([macro_abs_core_C, macro_abs_core_U], [1 - 1/N_CtoN_U, 1/N_CtoN_U])

macro_fiss_core = nf.macro(548, rho_C, molar_mass)
macro_fiss_core = nf.mixture([0, macro_fiss_core], [1 - 1/N_CtoN_U, 1/N_CtoN_U])

macro_scattering_U = nf.macro(15, rho_C, molar_mass)
macro_scattering_C = nf.macro(4.3, rho_C, molar_mass)
macro_scattering_core = nf.mixture([macro_scattering_C, macro_scattering_U], [1 - 1/N_CtoN_U, 1/N_CtoN_U])

sigma_tot = macro_abs_core + macro_fiss_core + macro_scattering_core
A_avg = nf.mixture([12.01, 235.044], [1 - 1/N_CtoN_U, 1/N_CtoN_U])
mu_avg = 2/(3*A_avg)
sigma_transport = 1/(sigma_tot*(1 - mu_avg))
extrapolation_distance = 0.7104/sigma_transport

In [28]:
################################################################################
# REGION DEFINITION
################################################################################
R_core = sp.symbols('R_out', positive = True) # Core radius
R_inner = R_core/2 # case2: half width
Core = Region(
    Diff=0.9, # Diffusion coefficient in the core
    Abs=macro_abs_core, 
    Fiss=macro_fiss_core, 
    Nu=2.44, # Average neutrons per fission in U-235
    Pos1=R_inner, 
    Pos2=R_core,
    composition='c', 
    power=0
)

In [29]:
################################################################################
# FUNCTION AND SYMBOLIC VARIABLES DEFINITION
################################################################################
r = sp.symbols('r')
B = sp.symbols('B', positive = True) 
L_c = sp.symbols('L_c', positive = True)
L_r = sp.symbols('L_r', positive = True)
D = sp.symbols('D', positive = True) # Sphere diffusion coefficient
R_core = sp.symbols('R_out', positive = True) # Core radius
F = sp.Function('phi', real = True)(r)


# Full sphere

In [30]:
# SPHERICAL CORE WITH NOTHING INSIDE
sC1 = sp.symbols('C_1')
flux_c = sp.Function('phi_c')(r)
flux_c = sp.Eq(
    flux_c,
    (sC1 * sp.sin(B * r)) / r
)
print('KNOWN SOLUTION FOR THE FLUX IN A SPHERE:')
display(flux_c)


KNOWN SOLUTION FOR THE FLUX IN A SPHERE:


Eq(phi_c(r), C_1*sin(B*r)/r)

In [31]:
# Vacuum boundary condition at the core boundary
flux_c = sp.Eq(flux_c.rhs.subs(r, R_core), 0)
display(flux_c)

Eq(C_1*sin(B*R_out)/R_out, 0)

In [32]:
# Compute the buckling
buckling = np.sqrt((Core.Nu * Core.Fission - Core.Absorption) / Core.Diffusion)
print('BUCKLING:', buckling)

# Substitute the buckling in the flux equation
flux_c = flux_c.subs(B, buckling)
display(flux_c)

BUCKLING: 0.07863735625744234


Eq(C_1*sin(0.0786373562574423*R_out)/R_out, 0)

In [33]:
# Find R_out
R_out = sp.solve(flux_c, R_core)[0] - extrapolation_distance
display(R_out)

39.6977923446823

In [34]:
# Compute the mass of the sphere
Volume = 4/3 * sp.pi * (R_out)**3
mass = rho_C * Volume / 1000
display(mass.evalf())

445.488602728747

The sphere is critical when it is 39.95cm in radius and weights 454Kg.

# Sphere with Vacuum inside

In [35]:
# ODE + BC FOR THE SPHERE WITH VACUUM INSIDE 

ODE = sp.Eq((1/r**2) * sp.diff(r**2 * sp.diff(F, r), r) + B**2 * F, 0)
print('ODE FOR THE SPHERE:')
display(ODE)

ODE FOR THE SPHERE:


Eq(B**2*phi(r) + (r**2*Derivative(phi(r), (r, 2)) + 2*r*Derivative(phi(r), r))/r**2, 0)

In [36]:
flux_c = sp.dsolve(ODE)
sC1 = sp.symbols('C_1', real = True)   
sC2 = sp.symbols('C_2', real = True)
flux_c = sp.Eq(
    flux_c.lhs,
    (sC1 * sp.sin(B * r)) / r + (sC2 * sp.cos(B * r)) / r
)
print('GENERAL SOLUTION FOR THE FLUX IN THE CORE (r /= 0):')
display(flux_c)

GENERAL SOLUTION FOR THE FLUX IN THE CORE (r /= 0):


Eq(phi(r), C_1*sin(B*r)/r + C_2*cos(B*r)/r)

In [37]:
# Vacuum BC 
vacuum_outside = sp.Eq(
    flux_c.rhs.subs(r, R_core),
    0
)
print('OUTSIDE VACUUM BOUNDARY CONDITION:')
display(vacuum_outside)

vacuum_inside = sp.Eq(
    -D * sp.diff(flux_c.rhs, r).subs(r, R_inner),
    0
)
print('INSIDE VACUUM BOUNDARY CONDITION:')
display(vacuum_inside)

OUTSIDE VACUUM BOUNDARY CONDITION:


Eq(C_1*sin(B*R_out)/R_out + C_2*cos(B*R_out)/R_out, 0)

INSIDE VACUUM BOUNDARY CONDITION:


Eq(-D*(2*B*C_1*cos(B*R_out/2)/R_out - 2*B*C_2*sin(B*R_out/2)/R_out - 4*C_1*sin(B*R_out/2)/R_out**2 - 4*C_2*cos(B*R_out/2)/R_out**2), 0)

In [38]:
# Substitute the buckling in the equations
system = [flux_c, vacuum_outside, vacuum_inside]
for i, eq in enumerate(system):
    system[i] = eq.subs(B, buckling).subs(D, Core.Diffusion)
    display(system[i])

Eq(phi(r), C_1*sin(0.0786373562574423*r)/r + C_2*cos(0.0786373562574423*r)/r)

Eq(C_1*sin(0.0786373562574423*R_out)/R_out + C_2*cos(0.0786373562574423*R_out)/R_out, 0)

Eq(-0.141547241263396*C_1*cos(0.0393186781287212*R_out)/R_out + 3.6*C_1*sin(0.0393186781287212*R_out)/R_out**2 + 0.141547241263396*C_2*sin(0.0393186781287212*R_out)/R_out + 3.6*C_2*cos(0.0393186781287212*R_out)/R_out**2, 0)

In [39]:
# Move all the terms with C1 on the lhs and C2 on the rhs

equation = system[1]

def rearrange_equation(equation):
    # Get the equation terms
    terms = equation.lhs.as_ordered_terms()

    # Get only the terms with C1
    lhs = sum([term for term in terms if term.has(sC1)])

    # Get only the terms with C1
    rhs = sum([term for term in terms if term.has(sC2)])
    
    # Create the new rearranged equation
    new_equation = sp.Eq(lhs,rhs, evaluate = False)
    return new_equation

eq_1 = rearrange_equation(system[1])
eq_2 = rearrange_equation(system[2])
display(eq_1)
display(eq_2)



Eq(C_1*sin(0.0786373562574423*R_out)/R_out, C_2*cos(0.0786373562574423*R_out)/R_out)

Eq(-0.141547241263396*C_1*cos(0.0393186781287212*R_out)/R_out + 3.6*C_1*sin(0.0393186781287212*R_out)/R_out**2, 0.141547241263396*C_2*sin(0.0393186781287212*R_out)/R_out + 3.6*C_2*cos(0.0393186781287212*R_out)/R_out**2)

In [40]:
# Ratio between the last two equations
ratio = sp.Eq(eq_1.lhs / eq_2.lhs, eq_1.rhs / eq_2.rhs)
ratio = sp.simplify(ratio)
ratio = sp.trigsimp(ratio)
display(ratio)

Eq(R_out*cos(0.0786373562574423*R_out)/(0.141547241263396*R_out*sin(0.0393186781287212*R_out) + 3.6*cos(0.0393186781287212*R_out)), -R_out*sin(0.0786373562574423*R_out)/(0.141547241263396*R_out*cos(0.0393186781287212*R_out) - 3.6*sin(0.0393186781287212*R_out)))

In [41]:
# Solve it numerically
sol = sp.nsolve(ratio, [40]) - extrapolation_distance # We use the first value computed analytically as the first guess
display(sol)

51.3452213303736

In [42]:
# Compute the volume
V = 4/3 * sp.pi * (sol)**3 - 4/3 * sp.pi * (sol/2)**3
mass = rho_C * V / 1000
display(mass.evalf())

843.423455550057

In the case of a hollow sphere the critical radius is 51.60cm (with a hole of radius= 25.8cm).  
The mass of the hollow sphere is 855 Kg

# Hollow sphere with absorber inside

In [43]:
# ODE + BC FOR THE SPHERE WITH PERFECTLY ABSORBING MEDIUM INSIDE 

ODE = sp.Eq((1/r**2) * sp.diff(r**2 * sp.diff(F, r), r) + B**2 * F, 0)
print('ODE FOR THE SPHERE:')
display(ODE)

ODE FOR THE SPHERE:


Eq(B**2*phi(r) + (r**2*Derivative(phi(r), (r, 2)) + 2*r*Derivative(phi(r), r))/r**2, 0)

In [44]:
flux_c = sp.dsolve(ODE)
sC1 = sp.symbols('C_1')   
sC2 = sp.symbols('C_2')
flux_c = sp.Eq(
    flux_c.lhs,
    (sC1 * sp.sin(B * r)) / r + (sC2 * sp.cos(B * r)) / r
)
print('GENERAL SOLUTION FOR THE FLUX IN THE CORE (r /= 0):')
display(flux_c)

GENERAL SOLUTION FOR THE FLUX IN THE CORE (r /= 0):


Eq(phi(r), C_1*sin(B*r)/r + C_2*cos(B*r)/r)

In [45]:
# Vacuum BC 
vacuum_outside = sp.Eq(
    flux_c.rhs.subs(r, R_core),
    0
)
print('OUTSIDE VACUUM BOUNDARY CONDITION:')
display(vacuum_outside)
abs_medium_inside = sp.Eq(
    (flux_c.rhs.subs(r,R_inner) / 4) - (D/2) * sp.diff(flux_c.rhs, r).subs(r, R_inner),
    0
)
print('PERFECTLY ABSORBING MEDIUM INSIDE BOUNDARY CONDITION:')
display(abs_medium_inside)

OUTSIDE VACUUM BOUNDARY CONDITION:


Eq(C_1*sin(B*R_out)/R_out + C_2*cos(B*R_out)/R_out, 0)

PERFECTLY ABSORBING MEDIUM INSIDE BOUNDARY CONDITION:


Eq(C_1*sin(B*R_out/2)/(2*R_out) + C_2*cos(B*R_out/2)/(2*R_out) - D*(2*B*C_1*cos(B*R_out/2)/R_out - 2*B*C_2*sin(B*R_out/2)/R_out - 4*C_1*sin(B*R_out/2)/R_out**2 - 4*C_2*cos(B*R_out/2)/R_out**2)/2, 0)

In [46]:
# Substitute the buckling in the equations
system = [flux_c, vacuum_outside, abs_medium_inside]
for i, eq in enumerate(system):
    system[i] = eq.subs(B, buckling).subs(D, Core.Diffusion)
    display(system[i])

Eq(phi(r), C_1*sin(0.0786373562574423*r)/r + C_2*cos(0.0786373562574423*r)/r)

Eq(C_1*sin(0.0786373562574423*R_out)/R_out + C_2*cos(0.0786373562574423*R_out)/R_out, 0)

Eq(C_1*sin(0.0393186781287212*R_out)/(2*R_out) - 0.0707736206316981*C_1*cos(0.0393186781287212*R_out)/R_out + 1.8*C_1*sin(0.0393186781287212*R_out)/R_out**2 + 0.0707736206316981*C_2*sin(0.0393186781287212*R_out)/R_out + C_2*cos(0.0393186781287212*R_out)/(2*R_out) + 1.8*C_2*cos(0.0393186781287212*R_out)/R_out**2, 0)

In [47]:
# Move all the terms with C1 on the lhs and C2 on the rhs
# we use the previously defined function :D
eq_1 = rearrange_equation(system[1])
eq_2 = rearrange_equation(system[2])
display(eq_1)
display(eq_2)

Eq(C_1*sin(0.0786373562574423*R_out)/R_out, C_2*cos(0.0786373562574423*R_out)/R_out)

Eq(C_1*sin(0.0393186781287212*R_out)/(2*R_out) - 0.0707736206316981*C_1*cos(0.0393186781287212*R_out)/R_out + 1.8*C_1*sin(0.0393186781287212*R_out)/R_out**2, 0.0707736206316981*C_2*sin(0.0393186781287212*R_out)/R_out + C_2*cos(0.0393186781287212*R_out)/(2*R_out) + 1.8*C_2*cos(0.0393186781287212*R_out)/R_out**2)

In [48]:
# Ratio between the last two equations
ratio = sp.Eq(eq_1.lhs / eq_2.lhs, eq_1.rhs / eq_2.rhs)
ratio = sp.simplify(ratio)
ratio = sp.trigsimp(ratio)
display(ratio)

Eq(2*R_out*cos(0.0786373562574423*R_out)/(R_out*(0.141547241263396*sin(0.0393186781287212*R_out) + cos(0.0393186781287212*R_out)) + 3.6*cos(0.0393186781287212*R_out)), 2*R_out*sin(0.0786373562574423*R_out)/(R_out*(sin(0.0393186781287212*R_out) - 0.141547241263396*cos(0.0393186781287212*R_out)) + 3.6*sin(0.0393186781287212*R_out)))

In [49]:
# Solve it numerically
sol = sp.nsolve(ratio, [51]) - extrapolation_distance # We use the second value as the first guess
display(sol)

76.2307259779295

In [50]:
# Compute the volume
V = 4/3 * sp.pi * (sol)**3 - 4/3 * sp.pi * (sol/2)**3
mass = rho_C * V / 1000
display(mass.evalf())

2760.16948982110

The sphere with absorber inside has a critical radius of 76.5cm and a mass of 2.78 tons